In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras import applications
from keras.optimizers import SGD,Adam
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='1'

In [3]:
train = pd.read_csv('aug/train.csv')

In [5]:
train.head()

,images,category
0,2812765.jpg,1
1,2792364.jpg,1
2,list1_4885.jpg,1
3,2841613.jpg,1
4,2823206.jpg,1


In [7]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('aug/images/'+train['images'][i], target_size=(224,224,1))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 27657/27657 [00:35<00:00, 776.05it/s]


In [8]:
y=train['category'].values
y = to_categorical(y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [10]:
def res_models(img_rows, img_cols, channel=1, num_classes=None):
    model = applications.resnet50.ResNet50(include_top=True, weights='imagenet')

    # Truncate and replace softmax layer for transfer learning
    
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    
    new_model = Sequential()
    new_model.add(model)
    new_model.add(Dense(num_classes, activation='softmax'))

#     Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:5]:
        layer.trainable = True

    # Learning rate is changed to 0.001
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return new_model

In [12]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 1
num_classes = 6
batch_size = 1
nb_epoch = 15

In [13]:
model = res_models(img_rows, img_cols, channel, num_classes)

In [17]:
# Start Fine-tuning
model.fit(X, y,
          nb_epoch=15,
          shuffle=True,
          verbose=1,
          validation_split=0.1
          )

/home/humonics/.virtualenvs/capi/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 24891 samples, validate on 2766 samples
Epoch 1/15
24891/24891 [==============================] - 210s 8ms/step - loss: 0.2440 - acc: 0.9157 - val_loss: 0.8290 - val_acc: 0.7169
Epoch 2/15
24891/24891 [==============================] - 204s 8ms/step - loss: 0.0392 - acc: 0.9891 - val_loss: 0.2520 - val_acc: 0.9114
Epoch 3/15
24891/24891 [==============================] - 205s 8ms/step - loss: 0.0111 - acc: 0.9974 - val_loss: 0.2511 - val_acc: 0.9237
Epoch 4/15
24891/24891 [==============================] - 205s 8ms/step - loss: 0.0053 - acc: 0.9988 - val_loss: 0.8187 - val_acc: 0.7928
Epoch 5/15
24891/24891 [==============================] - 205s 8ms/step - loss: 0.0054 - acc: 0.9985 - val_loss: 0.6881 - val_acc: 0.8265
Epoch 6/15
24891/24891 [==============================] - 205s 8ms/step - loss: 0.0041 - acc: 0.9991 - val_loss: 0.4895 - val_acc: 0.8677
Epoch 7/15
24891/24891 [==============================] - 205s 8ms/step - loss: 0.0035 - acc: 0.9992 - val_loss: 0.8833 - v

In [18]:
model.save_weights('final1.hdf5')
print("model saved..!")

model saved..!


In [19]:
test = pd.read_csv('test_ApKoW4T.csv')

In [22]:
test.head()

,image
0,1007700.jpg
1,1011369.jpg
2,1051155.jpg
3,1062001.jpg
4,1069397.jpg


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total params: 23,600,006
Trainable params: 23,546,886
Non-trainable params: 53,120
_________________________________________________________________


In [24]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('train/images/'+test['image'][i], target_size=(224,224,1))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 2680/2680 [00:05<00:00, 484.69it/s]


In [25]:
feature = model.predict(test)

In [26]:
results = []
for i in range(feature.shape[0]):
    pred = np.argmax(feature[i])
    results.append(pred)

In [27]:
test = pd.read_csv('test_ApKoW4T.csv')

In [28]:
sample = pd.read_csv('sample_submission_ns2btKE.csv')
sample['image'] = test['image']
sample['category'] = results
sample.to_csv('final1.csv', header=True, index=False)